<a href="https://colab.research.google.com/github/Ahmed-Shatla/Segmentation/blob/main/Segmentation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

**Hyperparameters**

In [ ]:
batch_size = 8
lr = 1e-4 #0.0001
epochs = 100
heights = 512
width = 512

**Building UNET**

conv block

In [ ]:
def conv_block(inputs,num_filters):
  x = Conv2D(num_filters,3,padding="same")(inputs)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  x = Conv2D(num_filters,3,padding="same")(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  return x

Encoder

In [ ]:
def encoder_block(inputs,num_filters):
  x = conv_block(inputs, num_filters)
  p = MaxPooling2D((2,2))(x)
  return x,p

Decoder

In [ ]:
def decoder_block(inputs,skip,num_filters):
  x = Conv2DTranspose(num_filters,(2,2),strides=2,padding="same") (inputs)
  x = Concatenate()([x,skip])
  x = conv_block(x,num_filters)
  return x

UNET


In [ ]:
def build_unet(input_shape):
  inputs = Input(input_shape)

  #Encoder#
  s1,p1 = encoder_block(inputs,64)
  s2,p2 = encoder_block(p1,128)
  s3,p3 = encoder_block(p2,256)
  s4,p4 = encoder_block(p3,512)

  #bridge
  b1 = conv_block(p4,1024)

  #Decoder
  d1 = decoder_block(b1,s4,512)
  d2 = decoder_block(d1,s3,256)
  d3 = decoder_block(d2,s2,128)
  d4 = decoder_block(d3,s1,64)

  outputs = Conv2D(1,1,padding="same",activation = "sigmoid")(d4)

  model = Model(inputs,outputs,name="UNET")
  return model

Load Data

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

img_gen=ImageDataGenerator(rescale=1/255)
msk_gen=ImageDataGenerator(rescale=1/255)

In [ ]:
from pathlib import Path
data_path = Path('/content/drive/MyDrive/Segmantation/Train')
img_ds = img_gen.flow_from_directory(
    data_path,
    target_size=(heights, width),
    batch_size=8,
    shuffle=True,
    class_mode=None,
    classes=['images']
)
msks_ds = msk_gen.flow_from_directory(
    data_path,
    target_size=(heights, width),
    batch_size=8,
    shuffle=False,
    class_mode=None,
    classes=['masks']
)
train_generator = zip(img_ds, msks_ds)




data_path = Path('/content/drive/MyDrive/Segmantation/Valid')
img_ds = img_gen.flow_from_directory(
    data_path,
    target_size=(heights, width),
    batch_size=8,
    shuffle=True,
    class_mode=None,
    classes=['images']
)

msks_ds = msk_gen.flow_from_directory(
    data_path,
    target_size=(heights, width),
    batch_size=8,
    shuffle=False,
    class_mode=None,
    classes=['masks']
)

valid_generator = zip(img_ds, msks_ds)


Found 15 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.


In [ ]:
input_shape = (heights,width,3)
model = build_unet(input_shape)

In [ ]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 512, 512, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 512, 512, 64)         0         ['batch_normalization[0][0]

In [ ]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["acc"])

In [ ]:
model.fit(train_generator,validation_data=valid_generator,epochs=epochs,callbacks=EarlyStopping(monitor='val_loss',patience=20))

Epoch 1/100
    581/Unknown - 1459s 2s/step - loss: 0.3081 - acc: 0.8314

KeyboardInterrupt: ignored